Created on Fri Jan 17 17:03:35 2019  
Group 9  
@authors:  

In [1]:
import pandas as pd
import numpy as np
import nltk
import json
from simpletransformers.ner import NERModel
from gensim.models import KeyedVectors
from strsimpy.jaro_winkler import JaroWinkler
from spellchecker import SpellChecker
from nltk.corpus import stopwords

In [2]:
%run constants.ipynb
%run functions.ipynb
%run dictionnaries.ipynb

In [3]:
%run tag_to_filter.v1.ipynb

In [4]:
df_transitions = pd.read_csv(bdd_directory+'df_transitions.csv', 
                             sep='§', 
                             engine='python',
                             index_col=0, 
                             encoding='utf-8')
df_transitions.shape

(10, 10)

In [5]:
class Markov:
    def __init__(self, name, transitions):
        self.name = name
        self.transitions = transitions
        
    def predict(self, event):
        predict = predict_next_state(event, self.transitions)
        return predict
    
    def random_state(self):
        return np.random.choice(self.transitions.index)
    
markov = Markov('Markov', df_transitions)
markov.predict(markov.random_state())

'{"filters": {"manufacturer": ["airbus"], "aircraft": ["a320"], "company": ["air france"], "country": ["england", "france"], "category": [], "date": ["01092017", "30092017"]}, "tab": "seat"}'

### Loads NER and W2V models 
source for the NER model : https://ufile.io/ichyycfe  
source for the W2V model : https://github.com/eyaler/word2vec-slim/blob/master/GoogleNews-vectors-negative300-SLIM.bin.gz  

In [6]:
all_tags = get_all_tags()
tagger = NERModel(model_type = 'bert',
                  model_name = data_directory+'bert/current_model/',
                  labels = all_tags,
                  use_cuda = False)

model_w2v = KeyedVectors.load_word2vec_format(pathword2vec, binary=True)
voc_stopwords = set(stopwords.words('english'))
db = get_BDD()

In [7]:
class Intent:
    def __init__(self, name, state=None):
        if state is None: state = json.dumps(init_event(tab=CT_tabs_default))
        self.name = name
        self.interface_in = state
        self.interface_out = None
        
    def synchronize(self, event):
        self.interface_in = event
        self.interface_out = None
    
    def get_interact_interface(self):
        return self.interface_out
        
    def interact(self, sentence):
        return "Please Implement this method"

class Displayer(Intent):
    def __init__(self, 
                 name, 
                 model_tagger, 
                 model_predicteur, 
                 model_w2v, 
                 db={}, 
                 voc_stopwords=set(),
                 state=None):
        
        super(Displayer, self).__init__(name)
        self.tagger = model_tagger
        self.predicteur = model_predicteur
        self.w2v = model_w2v
        self.voc_stopwords = voc_stopwords
        self.db = db
        
    def synchronize(self, event):
        self.interface_in = event
        self.interface_out = self.predicteur.predict(self.interface_in)
    
    def get_tags(self, sentence):
        sentence_corrected_list = self.auto_correction(sentence)
        sentence_corrected = " ".join(sentence_corrected_list)
        request = self.tagger.predict([sentence_corrected])[0][0]
        print(request,'\n')
        return request
    
    def get_filters(self, sentence):
        request = self.get_tags(sentence)
        tags_values = extract_tags(request)
        filters = tag_to_filters(tags_values)
        filters = apply_date(filters)
        event = {
            CT_tabs : CT_tabs_default,
            CT_filt : filters,
        }
        return event
    
    def get_output_sentence(self, pred_state_hash):
        pred_sentence = make_sentence_fom_json(json.loads(pred_state_hash))
        return pred_sentence
    
    def interact(self, sentence):
        event = self.get_filters(sentence)
        event_state = json.dumps(event)
        self.synchronize(json_string_to_hash(event_state))
        
        pred_state_hash = self.get_interact_interface()
        pred_sentence = self.get_output_sentence(pred_state_hash)
        
        res = pred_sentence
        return res
    
    def auto_correction(self, sentence):
        return auto_correction(self.w2v.vocab, db, voc_stopwords, sentence)
    
Intent_displayer = Displayer(name='Displayer_Recommandation',
                           model_tagger=tagger,
                           model_predicteur=markov,
                           model_w2v=model_w2v,
                           voc_stopwords=voc_stopwords,
                           db=db)
Intent_displayer.name

'Displayer_Recommandation'

In [8]:
test_event = Intent_displayer.interface_in
test_event

'{"tab": "general", "filters": {"manufacturer": [], "aircraft": [], "company": [], "country": [], "category": [], "date": []}}'

In [9]:
Intent_displayer.synchronize(test_event)
Intent_displayer.get_interact_interface()

'{"filters": {"manufacturer": ["airbus"], "aircraft": ["a350-900"], "company": [], "country": ["england", "france", "germany", "italy", "spain"], "category": [], "date": ["01092017", "30092017"]}, "tab": "general"}'

In [10]:
class Classifieur:
    def __init__(self, name, model_classifieur):
        self.name = name
        self.classifieur = model_classifieur
        
    def predict(self, sentence):
        return 0

Classifieur = Classifieur(name='Intent classifieur', model_classifieur="")
Classifieur.predict(sentence="")

0

In [11]:
intents  = {"classifieur" : Classifieur,
            "intents" : {0 : Intent_displayer}
           }

intents

{'classifieur': <__main__.Classifieur at 0x7f599fcebc18>,
 'intents': {0: <__main__.Displayer at 0x7f599fc8aac8>}}

In [12]:
class ChatBot:
    def __init__(self, name, intents={}, classifieur=""):
        self.name = name
        self.classifieur = intents['classifieur']
        self.intents = intents['intents']
        self.active_intent = list(intents.keys())[0]
    
    def classify(self, sentence):
        self.active_intent = self.classifieur.predict(sentence)
    
    def interact(self, sentence):
        self.classify(sentence)
        intent = self.intents[self.active_intent]
        interaction = intent.interact(sentence)
        return interaction
    
    def synchronize(self, event):
        for intent in self.intents.values():
            intent.synchronize(event)
    
    def get_interact_interface(self):
        return self.intents[self.active_intent].get_interact_interface()

Hubert = ChatBot(name='Chatbot Hubert', intents=intents)
Hubert.classify('aertetreb')
Hubert.synchronize(test_event)

In [13]:
%run functions.ipynb

output_sentence = Hubert.interact('show me the numberd of flight in FRAnCE for Aribus')
output_interfac = Hubert.get_interact_interface()

print(output_sentence, '\n')
print(output_interfac, '\n')

Converting to features started.



[{'show': 'O'}, {'me': 'O'}, {'the': 'O'}, {'number': 'STUDIED_B'}, {'of': 'STUDIED_I'}, {'flight': 'STUDIED_E'}, {'in': 'O'}, {'France': 'COUN_B'}, {'for': 'O'}, {'Airbus': 'MANU_B'}] 

We suggest you the seats' study from 01-01-2017 to 31-12-2017 for the manufacturer airbus in france. If you agree, click on the following link ;) 

{"filters": {"manufacturer": ["airbus"], "aircraft": [], "company": [], "country": ["france"], "category": [], "date": ["01012017", "31122017"]}, "tab": "seat"} 

